<h1>DOWNTUBE</h1>

In [20]:
import inquirer
from pyfiglet import Figlet
from pytube.__main__ import YouTube
from rich import print
from pytube import Stream
import time
import sys
import subprocess
import shutil


In [21]:
figlet = Figlet(font='big')
print(f"[red]{figlet.renderText('Downtube')}[/red]")

 _____                      _         _          
|  __ \                    | |       | |         
| |  | | _____      ___ __ | |_ _   _| |__   ___ 
| |  | |/ _ \ \ /\ / / '_ \| __| | | | '_ \ / _ \
| |__| | (_) \ V  V /| | | | |_| |_| | |_) |  __/
|_____/ \___/ \_/\_/ |_| |_|\__|\__,_|_.__/ \___|
                                                 
                                                 

In [22]:
import os
default_videos_folder = os.path.expanduser("~/Videos")
folder_name = "DownTube"
temp_name = "Temp"
video_name = "Video"
audio_name = "Audio"
subtitle_name = "Subtitle"
download_name = "Downloads"
# Construct the full path

full_path = os.path.join(default_videos_folder, folder_name)

# Check if the folder exists
if not os.path.exists(full_path):
    try:
        os.makedirs(full_path)
        print(f"Folder '{folder_name}' created successfully at {full_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

temp_path = os.path.join(full_path, temp_name)  # Create a temporary directory within the main one
if not os.path.exists(temp_path):
    try:
        os.makedirs(temp_path)
        print(f"Folder '{temp_name}' created successfully at {temp_path}")
        subprocess.run(["attrib","+H",temp_path],check=True)
    except Exception as e:
        print(f"An error occurred: {e}")

video_path = os.path.join(temp_path,video_name)
if not os.path.exists(video_path):
    try:
        os.makedirs(video_path)
        print(f"Folder '{video_name}' created successfully at {video_path}")
        # subprocess.run(["attrib","+H","myfile.txt"],check=True)
    except Exception as e:
        print(f"An error occurred: {e}")

audio_path = os.path.join(temp_path,audio_name)
if not os.path.exists(audio_path):
    try:
        os.makedirs(audio_path)
        print(f"Folder '{audio_name}' created successfully at {audio_path}")
        # subprocess.run(["attrib","+H","myfile.txt"],check=True)
    except Exception as e:
        print(f"An error occurred: {e}")

subtitle_path = os.path.join(temp_path,subtitle_name)
if not os.path.exists(subtitle_path):
    try:
        os.makedirs(subtitle_path)
        print(f"Folder '{subtitle_name}' created successfully at {subtitle_path}")
        # subprocess.run(["attrib","+H","myfile.txt"],check=True)
    except Exception as e:
        print(f"An error occurred: {e}")

In [23]:
video_url = "https://youtu.be/MwtKJG_87fw?si=ysG17gJK52FsFny9"
url = 'https://youtu.be/gh3FyLT7WVg?list=RDgh3FyLT7WVg'
url_supa = 'https://youtu.be/dU7GwCOgvNY?si=ovCJGbiQKdaFoVXm'

In [24]:
yt = YouTube(url=url)
video = yt.streams.filter(only_video=True)
audio = yt.streams.filter(only_audio=True)

In [25]:
resolution = dict()
for x in video:
        resolution[x.itag] = [x.resolution,x.fps,x.filesize_mb,x.mime_type]

In [26]:
resolution = dict()
for x in audio:
        resolution[x.itag] = [x.mime_type,x.abr,x.filesize_mb]

In [27]:
resolution

{139: ['audio/mp4', '48kbps', 1.645],
 140: ['audio/mp4', '128kbps', 4.362],
 249: ['audio/webm', '50kbps', 1.711],
 250: ['audio/webm', '70kbps', 2.226],
 251: ['audio/webm', '160kbps', 4.339]}

In [28]:
highest_quality_stream = None
highest_bitrate = 0
for stream in audio:
    if stream.type == "audio":
        bitrate = int(stream.abr.replace("kbps", ""))
        if bitrate > highest_bitrate:
            highest_bitrate = bitrate
            highest_quality_stream = stream

In [29]:
highest_quality_stream

<Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">

In [30]:
start_time = time.time()
def display_progress_bar(
    bytes_received: int, filesize: int, ch: str = "▬", scale: float = 0.55
) -> None:
    """Display a simple, pretty progress bar.

    Example:
    ~~~~~~~~
    PSY - GANGNAM STYLE(강남스타일) MV.mp4
    ↳ |▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬| 100.0%

    :param int bytes_received:
        The delta between the total file size (bytes) and bytes already
        written to disk.
    :param int filesize:
        File size of the media stream in bytes.
    :param str ch:
        Character to use for presenting progress segment.
    :param float scale:
        Scale multiplier to reduce progress bar size.

    """
    columns = shutil.get_terminal_size().columns
    max_width = int(columns * scale)

    filled = int(round(max_width * bytes_received / float(filesize)))
    remaining = max_width - filled
    progress_bar = ch * filled + " " * remaining
    # 
    downloaded_size = f"{bytes_received / (1024 * 1024):.2f}MB"
    total_size = f"{filesize / (1024 * 1024):.2f}MB"
    elapsed_time = time.time() - start_time
    download_speed = bytes_received / elapsed_time  # Bytes per second
    time_left = (filesize - bytes_received) / download_speed
    time_left_str = time.strftime("%H:%M:%S", time.gmtime(time_left))
    # 
    percent = round(100.0 * bytes_received / float(filesize), 1)
    text = f"↳|{progress_bar}| {percent}% - {downloaded_size}/{total_size} - {time_left_str} left\r"
    sys.stdout.write(text)
    sys.stdout.flush()


In [31]:
def on_progress(
    stream: Stream, chunk: bytes, bytes_remaining: int
) -> None:  # pylint: disable=W0613
    filesize = stream.filesize
    bytes_received = filesize - bytes_remaining
    display_progress_bar(bytes_received, filesize)



In [32]:
v_path = yt.streams.get_by_itag(248).download(output_path=video_path)


In [33]:
filename = yt.streams.get_by_itag(248).default_filename
filename

'Kurchi Madathapetti.webm'

In [34]:
v_path

'C:\\Users\\NEEVEN PRADHAPH/Videos\\DownTube\\Temp\\Video\\Kurchi Madathapetti.webm'

In [35]:
a_path = yt.streams.get_by_itag(251).download(output_path=audio_path)
a_path

'C:\\Users\\NEEVEN PRADHAPH/Videos\\DownTube\\Temp\\Audio\\Kurchi Madathapetti.webm'

In [36]:
from moviepy.editor import VideoFileClip, AudioFileClip
import traceback

def combine_video_audio(video_path, audio_path, output_path):
    try:
        # Load the video and audio files
        # video_clip = VideoFileClip(video_path)
        # audio_clip = AudioFileClip(audio_path)
        with VideoFileClip(video_path) as video_clip, AudioFileClip(audio_path) as audio_clip:

        # Get metadata of video file
            video_fps = video_clip.fps
            video_size = video_clip.size
            video_rotation = video_clip.rotation

            # Set the audio of the video clip to the loaded audio clip
            video_clip = video_clip.set_audio(audio_clip)

            # Write the combined video with audio to a new file
            video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

            # Close the clips
            video_clip.close()
            audio_clip.close()

    except KeyError as e:
        # Handle KeyError
        print(f"Error: {e}")
        print("Video metadata is missing required keys.")
        traceback.print_exc()  # Print traceback for debugging

if __name__ == "__main__":
    video_path = v_path
    audio_path = a_path
    output_path = full_path+'\\'+os.path.splitext(filename)[0]+'.mp4'

    combine_video_audio(video_path, audio_path, output_path)
    print(output_path)


Moviepy - Building video C:\Users\NEEVEN PRADHAPH/Videos\DownTube\Kurchi Madathapetti.mp4.
MoviePy - Writing audio in Kurchi MadathapettiTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video C:\Users\NEEVEN PRADHAPH/Videos\DownTube\Kurchi Madathapetti.mp4



t:   7%|▋         | 505/6780 [00:09<01:55, 54.10it/s, now=None]

KeyboardInterrupt: 

In [37]:
os.remove(v_path)
os.remove(a_path)

t:   8%|▊         | 510/6780 [00:25<01:55, 54.10it/s, now=None]